## Full fine-tuning EleutherAI/pythia-160m to predict

### Dataset

In [ ]:
!pip install transformers

In [3]:
!pip install datasets

In [21]:
import os
currentPath = os.getcwd().replace('\\','/')    # get current path

print(currentPath)

/home/Arthur/aicup


In [22]:
from datasets import load_dataset, Features, Value

dataset = load_dataset("csv", data_files="PublicDataset_phase3/merged_20231120(2).tsv", delimiter='\t',
                       features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['fid', 'idx', 'content', 'label'],
        num_rows: 138930
    })
})

In [4]:
dataset['train'][110]

{'fid': '102',
 'idx': 2408,
 'content': '-  320 mm benign leiomyoma, probably arising from the broad ligament.',
 'label': 'PHI:Null'}

In [5]:
dataset['train'][1]

{'fid': '10',
 'idx': 25,
 'content': '091016.NMT',
 'label': 'MEDICALRECORD:091016.NMT'}

In [9]:
dataset['train'][7]

{'fid': '10',
 'idx': 114,
 'content': 'D.O.B:  24/8/1993',
 'label': 'DATE: 24/8/1993=>1993-08-24'}

For demonstration purpose, we only use the randomly sampled 20000 instances.

In [24]:
import torch
# sub_datasets = torch.utils.data.random_split(dataset['train'], [20000, 65736])
sub_datasets2 = dataset['train']
#print(len(sub_datasets[0]))
#for i in range(4): print(sub_datasets[0][i])
print(len(sub_datasets2))
for i in range(4): print(sub_datasets2[i])

138930
{'fid': '10', 'idx': 1, 'content': 'Episode No:  09F016547J', 'label': 'IDNUM: 09F016547J'}
{'fid': '10', 'idx': 25, 'content': '091016.NMT', 'label': 'MEDICALRECORD: 091016.NMT'}
{'fid': '10', 'idx': 37, 'content': 'SIZAR, HOWARD', 'label': 'PATIENT: SIZAR, HOWARD'}
{'fid': '10', 'idx': 52, 'content': 'Lab No:  09F01654', 'label': 'IDNUM: 09F01654'}


### Data loader

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-160m" #"EleutherAI/pythia-70m-deduped"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

<|pad|>: 50278


In [4]:
!pip install islab-opendeid

In [26]:
from torch.utils.data import DataLoader
from islab.aicup import collate_batch_with_prompt_template

train_data = list(sub_datasets2)
train_dataloader = DataLoader(train_data, batch_size=3, shuffle=False, collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer))
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
next(iter(titer))

torch.Size([3, 23])


(tensor([[50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
          50278, 50278, 50278,     0, 10118,  1621,    27, 50276,  2693,    39,
            520, 29195, 50279,  1838, 20872,    27, 15630,    39,   520, 29195,
            209, 50277],
         [50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
          50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
              0, 14311,  4379, 50279,  1267,  1848,  2025,    27, 14311,  4379,
            209, 50277],
         [    0,   416,  1400, 42525, 50276,    53,  1719, 50276,  1235,  2759,
          50279,    36,  7400,    27,   416,  1400, 42525,    61,    79, 19247,
             27,   308,  1719,    61,    79,    59,  3123,    27,  2456,  2759,
            209, 50277]]),
 tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,     0, 10118,  1621,    27, 50276,  2693,    39,
            520, 29195, 50279,  1838, 20872

In [27]:
results = tokenizer(["Lab No: 14H02780", "“STOCKDALE” 653 MONAGHAN RD"], padding=True)
print(results['input_ids'])
print()
print(results['input_ids'][0])
print(tokenizer.decode(results['input_ids'][0]))
print(results['input_ids'][1])
print(tokenizer.decode(results['input_ids'][1]))

[[50278, 50278, 50278, 50278, 50278, 50278, 21663, 1621, 27, 1638, 41, 16604, 1438], [1628, 1267, 9466, 37, 23502, 668, 721, 3357, 33995, 2696, 41, 1539, 28613]]

[50278, 50278, 50278, 50278, 50278, 50278, 21663, 1621, 27, 1638, 41, 16604, 1438]


2023-12-02 13:25:03.431993: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-02 13:25:03.433069: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 13:25:03.447111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 13:25:03.447134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 13:25:03.447147: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>Lab No: 14H02780
[1628, 1267, 9466, 37, 23502, 668, 721, 3357, 33995, 2696, 41, 1539, 28613]
“STOCKDALE” 653 MONAGHAN RD


In [28]:
results = tokenizer(
    [f"{bos} 9364819.RAN\\nMINTANIA, JEFFRY {sep} ID: 9364819.RAN\\nNAME: MINTANIA, JEFFRY {eos}",
     f"{bos} This is a sentence {sep} PHI: NULL {eos}"],
    padding=True
)
print(results['attention_mask'][0])
print(results['attention_mask'][1])
print(tokenizer.decode(results['input_ids'][0]))
print(tokenizer.decode(results['input_ids'][1]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|endoftext|> 9364819.RAN\nMINTANIA, JEFFRY 

####

 ID: 9364819.RAN\nNAME: MINTANIA, JEFFRY <|END|>
<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|endoftext|> This is a sentence 

####

 PHI: NULL <|END|>


In [14]:
from islab.aicup import OpenDeidBatchSampler

BATCH_SIZE = 10
bucket_train_dataloader = DataLoader(train_data,
                                     batch_sampler=OpenDeidBatchSampler(train_data, BATCH_SIZE),
                                     collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                                     pin_memory=True)

# for idx, batch in enumerate(bucket_train_dataloader):
#     print(batch)
#     print(batch[0].shape)
#     print(batch[1].shape)
#     break

### Model

In [16]:
from transformers import AutoConfig
# the model config to which we add the special tokens
config = AutoConfig.from_pretrained(plm,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Use Schedualr CosineAnnealingLR

In [18]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

EPOCHS = 100  # CHANGE TO THE NUMBER OF EPOCHS YOU WANT
optimizer = AdamW(model.parameters(), lr=2e-5)  # YOU CAN ADJUST LEARNING RATE

# You may want to adjust the number of warmup steps based on your specific use case
total_steps = len(bucket_train_dataloader) * EPOCHS
warmup_steps = int(0.1 * total_steps)  # 10% of total steps for warmup

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

---

In [12]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

EPOCHS = 100 # CHANGE TO THE NUMBER OF EPOCHS YOU WANT
optimizer = AdamW(model.parameters(),lr=2e-5) # YOU CAN ADJUST LEARNING RATE

model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [19]:
from tqdm import tqdm, trange

global_step = 0
total_loss = 0

model.train()
for epoch in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    # Training loop
    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels, attention_mask=masks)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))


Epoch:   1%|          | 1/100 [05:35<9:14:23, 335.99s/it]

Average train loss: 2.5601930281843592


Epoch:   2%|▏         | 2/100 [11:11<9:08:36, 335.88s/it]

Average train loss: 1.4473368288339243


Epoch:   3%|▎         | 3/100 [16:48<9:03:22, 336.11s/it]

...

Average train loss: 0.5032372264254388


Epoch:  99%|█████████▉| 99/100 [9:14:58<05:35, 335.85s/it]

Average train loss: 0.502499226151981


Epoch: 100%|██████████| 100/100 [9:20:34<00:00, 336.35s/it]

Average train loss: 0.5017529559689524


In [13]:
from tqdm import tqdm,trange

global_step = 0
total_loss = 0

model.train()
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    # Training loop
    predictions , true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels, attention_mask=masks)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

Epoch:   1%|▏         | 1/80 [04:36<6:03:30, 276.08s/it]

Average train loss: 1.442842539335673


Epoch:   2%|▎         | 2/80 [09:12<5:58:48, 276.00s/it]

Average train loss: 1.1517535237822736


Epoch:   4%|▍         | 3/80 [13:47<5:54:09, 275.97s/it]

...

Average train loss: 0.5320236786312514


Epoch:  99%|█████████▉| 79/80 [6:03:30<04:36, 276.41s/it]

Average train loss: 0.5318413260266524


Epoch: 100%|██████████| 80/80 [6:08:07<00:00, 276.10s/it]

Average train loss: 0.5319153038941056


In [29]:
import torch
model = torch.load('./models/1201_final.pt')
model.eval() # start eval

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

### Read the final file to be predicted

In [30]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset("csv", data_files="final/opendid_test.tsv", delimiter='\t',
                          features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'])
valid_list= list(valid_data['train'])
valid_list

[{'fid': '1097', 'idx': 1, 'content': '433475.RDC', 'label': None},
 {'fid': '1097', 'idx': 12, 'content': 'Timmins, ELDEN', 'label': None},
 {'fid': '1097', 'idx': 27, 'content': '43J47561,43J47561', 'label': None},
 {'fid': '1097',
  'idx': 46,
  'content': 'Last edited : 7/9/2063  Page: 2',
  'label': None},
 {'fid': '1097', 'idx': 78, 'content': 'CLINICAL:', 'label': None},
 {'fid': '1097',
  'idx': 88,
  'content': 'Metastatic cancer ?colorectal primary.',
  'label': None},
 {'fid': '1097', 'idx': 128, 'content': 'MACROSCOPIC:', 'label': None},
 {'fid': '1097',
  'idx': 141,
  'content': 'Specimen labelled "Omentum secondary", consists of a piece of omentum 120 x 100 x 30mm.',
  'label': None},
 {'fid': '1097',
  'idx': 230,
  'content': 'On sectioning there are multiple fibrotic white ill-defined nodules identified.',
  'label': None},
 {'fid': '1097',
  'idx': 312,
  'content': 'Blocks: 1 to 5 - representative sections from the nodules.',
  'label': None},
 {'fid': '1097',
  'id

In [23]:
for entry in valid_list:
    if entry['content'] is None or entry['content'] == "":
        print(entry)
        break
else:
    print(" 'content' row have no Null value。")


 'content' 列中没有空值。


In [31]:
from tqdm import tqdm
from islab.aicup import aicup_predict
import io
BATCH_SIZE = 32

with open("./final/1202_ver2.txt",'w',encoding='utf8') as f:
#with io.open("answer.txt",'w',encoding='utf8') as f:
    for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
        with torch.no_grad():
            seeds = valid_list[i:i+BATCH_SIZE]
            outputs = aicup_predict(model, tokenizer, input=seeds)
            for o in outputs:
                f.write(o)
                f.write('\n')

100%|██████████| 2469/2469 [03:36<00:00, 11.43it/s]


In [20]:
model_path = r"models"

In [24]:
import os

In [17]:
import torch
# torch.save(model.state_dict(), os.path.join(model_path , 'GPT_best.pt'))
torch.save(model, os.path.join(model_path , '1201_final.pt'))

NameError: name 'model' is not defined

## Data post processing

In [32]:
f = open("./final/1202_ver2.txt", "r", encoding='utf8')
w = open("./final/finalsub_revised1202_ver2.txt", "w", encoding='utf8')

for line in f.readlines():
    if "Null" not in line.strip() and "NULL" not in line.strip():
        w.write(line)

f.close()
w.close()


In [33]:
#predicted answer
answer_path = "/home/Arthur/aicup/final/finalsub_revised1202_ver2.txt"
#test data tsv
tsv_path = "/home/Arthur/aicup/final/opendid_test.tsv"
#output
# w = open("/home/Arthur/aicup/answer/answer2", "w")
submission = "/home/Arthur/aicup/final/answer3.txt"

In [34]:
sub = open(submission, 'w', encoding="utf-8")
tsv = open(tsv_path, 'r', encoding='utf-8')

In [35]:
def detect(private, line):
  A_index = line.find(private[0])
  index = [A_index]

  #find first phi
  if A_index >= 0:
    for i in range(1, len(private)):
      index.append(line.find(private[i][:-1], index[i-1]+1))

  #can't find first phi
  else:
    for i in range(1, len(private)):
      index.append(line.find(private[i][:-1], index[i-1]+1))
  #print(private, index)
  return index

In [36]:
#tsv = open(tsv_path, 'r', encoding='utf-8')
phi = ['PATIENT', 'DOCTOR', 'USERNAME', 'PROFESSION', 'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER', 'AGE', 'DATE',
      'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDR', 'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT', 'LICENSE', 'VECHICLE', 'DEVICE', 'BIOID', 'IDNUM', 'OTHER']
time = ['DATE', 'TIME', 'DURATION', 'SET']
with open(answer_path, 'r', encoding="utf-8") as answer:
  tsv_line = tsv.readlines()
  tsv_index = 0
  #print(tsv_line)
  for line in answer.readlines():
    words = line.split('\t')
    file_name = words[0]
    phi_name = words[1]
    start = words[2]
    end = words[3]
    data = '\t'.join(words[4:])
    #print(line)
    #drop phi not included in rules
    if phi_name not in phi:
      continue

    #multiple phi in one line
    if '\\n' in data:
      counts = data.split('\\n')
      phi_type, phi_data, normalized = [phi_name], [], []

      if phi_name in time:
        time_split = counts[0].split('\t')
        phi_data.append(time_split[0])
        normalized.append('\t'.join(time_split[1:]).strip())
      else:
        phi_data.append(counts[0])

      for i in range(1, len(counts)):
        temp_phi = counts[i].split(':')[0]
        if temp_phi not in phi:
          continue
        if temp_phi in time:
          time_split = counts[i].split('\t')
          temp = ':'.join(time_split[0].split(':')[1:])
          normalized.append('\t'.join(time_split[1:]).strip())
        else:
          temp = ':'.join(counts[i].split(':')[1:])
        phi_type.append(temp_phi)
        phi_data.append(temp.strip())

      time_count = 0
      count = 0
      #tsv and answer not in the smae file
      while file_name > tsv_line[tsv_index].split('\t')[0]:
        tsv_index += 1
      while int(tsv_line[tsv_index].split('\t')[1]) < int(start):
        tsv_index += 1
      tsv_index -= 1
      temp_index = tsv_index
      while count < 3 and file_name == tsv_line[tsv_index].split('\t')[0]:
        split_tsv = tsv_line[tsv_index].split('\t')
        line_start = int(split_tsv[1])
        data_tsv = '\t'.join(split_tsv[2:])
        return_index = detect(phi_data, data_tsv)


        #can't find phi in this line
        if len(set(return_index)) == 1 and return_index[0] == -1:
          if int(tsv_line[tsv_index].split('\t')[1]) > int(start) and file_name == tsv_line[tsv_index].split('\t')[0]:
            #print(file_name,int(tsv_line[tsv_index].split('\t')[1]) , int(start))
            count += 1
          tsv_index += 1

        #find in this line
        else:
          #calculate start and end index
          split_tsv = tsv_line[tsv_index].split('\t')
          data_tsv = '\t'.join(split_tsv[2:])
          line_start = int(split_tsv[1])

          #print(line_start,phi_data)
          for i in range(len(return_index)):
            if return_index[i] < 0:
              continue
            temp = phi_data[i]
            phi_name = phi_type[i]
            index = return_index[i]
            if '"' in data_tsv:
              index -= 1
            start = line_start + index
            if '\n' in temp:
              end = start + len(temp) -1
              temp = temp[:-1]
            else:
              end = start + len(temp)
            #write into file
            if phi_name in time:
              sub.write('\t'.join([file_name, phi_name, str(start), str(end), temp, normalized[time_count]])+'\n')
              #print('\t'.join([file_name, phi_name, str(start), str(end), temp, normalized[time_count]]))
              time_count += 1
            else:
              sub.write('\t'.join([file_name, phi_name, str(start), str(end), temp])+'\n')
              #print('\t'.join([file_name, phi_name, str(start), str(end), temp]))
          tsv_index += 1
          break
      tsv_index = temp_index

    #no problem line
    else:
      #write into file
      sub.write('\t'.join([file_name, phi_name, str(start), str(end), data]))
      #print('\t'.join([file_name, phi_name, str(start), str(end), data]))


In [37]:
sub.close()
tsv.close()

In [39]:
def check_countries_in_text(file_path):
    # List of common country names
    countries = [
    "USA", "Japan", "China", "India", "Germany", "France", "Brazil", "Australia", "Canada", "United Kingdom",
    "Italy", "Russia", "South Korea", "Mexico", "Indonesia", "Turkey", "Netherlands", "Saudi Arabia", "Switzerland",
    "Spain", "Argentina", "Belgium", "Sweden", "Norway", "Austria", "Poland", "Iran", "Thailand", "Denmark", "South Africa",
    "Singapore", "Malaysia", "Ireland", "Israel", "Egypt", "Greece", "Philippines", "Finland", "New Zealand", "Portugal",
    "Pakistan", "Vietnam", "Colombia", "Chile", "Czech Republic", "Romania", "Hungary", "Ukraine", "Peru", "Belarus",
    "Norway", "Bangladesh", "Nigeria", "Kenya", "Morocco", "Venezuela", "Cuba", "Ecuador", "Norway", "Qatar",
    "Lebanon", "Algeria", "Tunisia", "Malta", "Kuwait", "Bahrain", "Oman", "Jordan", "Cyprus", "Sri Lanka",
    "Nepal", "Bhutan", "Mongolia", "Laos", "Cambodia", "Fiji", "Vanuatu", "Samoa", "Tonga", "Solomon Islands",
    "Palau", "Micronesia", "Marshall Islands", "Tuvalu", "Kiribati", "Comoros", "Seychelles", "Maldives", "Antarctica"
]

# You can use this expanded list in your Python script.


# You can use this expanded list in the previous script for checking countries in a text file.

    # Read the content of the text file
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text_content = file.read()

        # Check if any country name is present in the text
        found_countries = [country for country in countries if country in text_content]

        if found_countries:
            print("The following countries were found in the text:")
            for country in found_countries:
                print(country)
        else:
            print("No country names found in the text.")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


# Example usage: Replace 'your_text_file.txt' with the path to your text file
check_countries_in_text('/home/Arthur/aicup/final/opendid_test.tsv')


The following countries were found in the text:
USA
Australia
Israel
Chile
Laos


In [61]:
f = open("./answer/answer7.txt", "r")
w = open("./answer/answer8.txt", "w")



for line in f.readlines():
    # To split columns on each line, use split() instead of split('\t')
    fields = line.split()
    
    # Check whether the number of fields is 4, if not, write it into new file
    if len(fields) != 4:
        w.write(line)

f.close()
w.close()

